## BooksInfo Database Design
#### First Approach
* We Let MySQL to Auto-Increment catID as a Primary Key in TABLE Categories then match it as Foreign Key in TABLE Books

In [8]:
# Imports the necessary libraries.
import requests
import csv
from bs4 import BeautifulSoup
import json
import pandas as pd
from datetime import datetime
from sqlalchemy import create_engine
import mysql.connector

# URL of the website to be scraped
url = "https://books.toscrape.com/"

# Make a GET request to fetch the raw HTML content
response = requests.get(url)

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, "lxml")

# Get all the book categories from the sidebar
categories = soup.find('div', class_='side_categories').find_all('a')
category_urls = [url + category['href'] for category in categories[1:]]
# Adjust links to solve pagination
page_num = 1
# category_urls = [link[:-10] + f'page-{page_num}' + link[-5:] for link in category_urls]

category_links = []
for link in category_urls:
    try:
        response = requests.get(link)
        soup = BeautifulSoup(response.content, "lxml")
        soup.find('li', {'class':'current'}).text.split('of')[1].strip()
        category_links.append(link[:-10] + f'page-{page_num}' + link[-5:])
    except:
        category_links.append(link)

# Create an empty list to store the scraped data and pagination start
title = []
price = []
star = []
category = []
link = []

# Loop through each category and scrape the book details
for category_url in category_links:
    if 'page' in category_url:
        page_num = 1
        while True:
            category_url = category_url[:-6] + f'{page_num}' + category_url[-5:]
            result = requests.get(category_url)
            src = result.content
            soup = BeautifulSoup(src, "lxml")
            total_pages = int(soup.find('li', {'class':'current'}).text.split('of')[1].strip())
            
            categorys = soup.find('h1').text.strip()
            titles = soup.find_all('h3')
            prices = soup.find_all('p', {'class':'price_color'})
            stars = soup.find_all("article", class_="product_pod")
            links = soup.find_all('h3')

            for i in range(len(titles)):
                title.append(titles[i].find('a').get('title'))
                price.append(prices[i].text[1:])
                star.append(stars[i].select_one("p.star-rating").get("class")[1])
                category.append(categorys)
                link.append('https://books.toscrape.com/catalogue/' + links[i].find('a').get('href'))
            
            page_num += 1
            
            
            if (page_num > total_pages):
                #print(f'Done Scraping {categorys} From {page_num-1} Pages')
                break
    else:
        result = requests.get(category_url)
        src = result.content
        soup = BeautifulSoup(src, "lxml")
        categorys = soup.find('h1').text.strip()
        titles = soup.find_all('h3')
        prices = soup.find_all('p', {'class':'price_color'})
        stars = soup.find_all("article", class_="product_pod")
        links = soup.find_all('h3')

        for i in range(len(titles)):
            title.append(titles[i].find('a').get('title'))
            price.append(float(prices[i].text[1:]))
            star.append(stars[i].select_one("p.star-rating").get("class")[1])
            category.append(categorys)
            link.append('https://books.toscrape.com/catalogue/' + links[i].find('a').get('href'))
        # print(f'Done Scraping {categorys}')


# Set up the connection
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='UseYourPass :D'
)

# Create a new cursor
cursor = conn.cursor()

# Drop the database if it already exists
cursor.execute("DROP DATABASE IF EXISTS BooksInfo")
# Create the database
cursor.execute('CREATE DATABASE BooksInfo;')
cursor.execute("USE BooksInfo;")
cursor.execute("""CREATE TABLE Categories(
                catID INT NOT NULL UNIQUE AUTO_INCREMENT,
                catName VARCHAR(255),

                PRIMARY KEY(catID));""")
cursor.execute("""CREATE TABLE Books (
                bookID INT NOT NULL AUTO_INCREMENT,
                bookName VARCHAR(255) NOT NULL,
                bookPrice DECIMAL(10, 2) NOT NULL,
                bookRating INT,
                catID INT,

                PRIMARY KEY (bookID),
                FOREIGN KEY (catID) REFERENCES categories(catID));""")
query = "INSERT INTO Categories(catName) VALUES (%s)"
scategory = set(category)
for cat in scategory:
    cursor.execute(query, (cat,))
    conn.commit()
cursor.execute("""SELECT * FROM Categories""")
catDict = {item[1]: item[0] for item in cursor.fetchall()}
rating_map = {'One': 1, 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5}
star = [rating_map[r] for r in star]
catID = [catDict[item] for item in category]
# Combine the lists into a list of tuples
books = list(zip(title, price, star, catID))
for value in books:
    #print(value)
    cursor.execute('INSERT INTO Books (bookName, bookPrice, bookRating, catID) VALUES (%s, %s, %s, %s)', value)
    conn.commit()
query = """

            SELECT *
            FROM Books as b INNER JOIN Categories as c 
            ON b.catID = c.catID;


"""
display(pd.read_sql(query, conn))
# conn.close()

/var/folders/y4/h840pn3s4knbxtcdr02w0kj40000gn/T/ipykernel_6367/3540157881.py:146: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(query, conn)


,bookID,bookName,bookPrice,bookRating,catID,catName
0,12,Sharp Objects,47.82,4,1,Mystery
1,13,"In a Dark, Dark Wood",19.63,1,1,Mystery
2,14,The Past Never Ends,56.50,4,1,Mystery
3,15,A Murder in Time,16.64,1,1,Mystery
4,16,The Murder of Roger Ackroyd (Hercule Poirot #4),44.10,4,1,Mystery
...,...,...,...,...,...,...
995,316,Counting Thyme,10.62,1,50,Childrens
996,317,Are We There Yet?,10.66,3,50,Childrens
997,318,Diary of a Minecraft Zombie Book 1: A Scare of...,52.88,4,50,Childrens
998,319,Matilda,28.34,1,50,Childrens


#### Second Approach
* We Let Python to Auto-Increment catID Then USE it as a Primary Key in TABLE Categories then match it as Foreign Key in TABLE Books

In [9]:
# Imports the necessary libraries.
import requests
import csv
from bs4 import BeautifulSoup
import json
import pandas as pd
from datetime import datetime
from sqlalchemy import create_engine
import mysql.connector

# URL of the website to be scraped
url = "https://books.toscrape.com/"

# Make a GET request to fetch the raw HTML content
response = requests.get(url)

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, "lxml")

# Get all the book categories from the sidebar
categories = soup.find('div', class_='side_categories').find_all('a')
category_urls = [url + category['href'] for category in categories[1:]]
# Adjust links to solve pagination
page_num = 1
# category_urls = [link[:-10] + f'page-{page_num}' + link[-5:] for link in category_urls]

category_links = []
for link in category_urls:
    try:
        response = requests.get(link)
        soup = BeautifulSoup(response.content, "lxml")
        soup.find('li', {'class':'current'}).text.split('of')[1].strip()
        category_links.append(link[:-10] + f'page-{page_num}' + link[-5:])
    except:
        category_links.append(link)

# Create an empty list to store the scraped data and pagination start
title = []
price = []
star = []
category = []
link = []

# Loop through each category and scrape the book details
for category_url in category_links:
    if 'page' in category_url:
        page_num = 1
        while True:
            category_url = category_url[:-6] + f'{page_num}' + category_url[-5:]
            result = requests.get(category_url)
            src = result.content
            soup = BeautifulSoup(src, "lxml")
            total_pages = int(soup.find('li', {'class':'current'}).text.split('of')[1].strip())
            
            categorys = soup.find('h1').text.strip()
            titles = soup.find_all('h3')
            prices = soup.find_all('p', {'class':'price_color'})
            stars = soup.find_all("article", class_="product_pod")
            links = soup.find_all('h3')

            for i in range(len(titles)):
                title.append(titles[i].find('a').get('title'))
                price.append(prices[i].text[1:])
                star.append(stars[i].select_one("p.star-rating").get("class")[1])
                category.append(categorys)
                link.append('https://books.toscrape.com/catalogue/' + links[i].find('a').get('href'))
            
            page_num += 1
            
            
            if (page_num > total_pages):
                #print(f'Done Scraping {categorys} From {page_num-1} Pages')
                break
    else:
        result = requests.get(category_url)
        src = result.content
        soup = BeautifulSoup(src, "lxml")
        categorys = soup.find('h1').text.strip()
        titles = soup.find_all('h3')
        prices = soup.find_all('p', {'class':'price_color'})
        stars = soup.find_all("article", class_="product_pod")
        links = soup.find_all('h3')

        for i in range(len(titles)):
            title.append(titles[i].find('a').get('title'))
            price.append(float(prices[i].text[1:]))
            star.append(stars[i].select_one("p.star-rating").get("class")[1])
            category.append(categorys)
            link.append('https://books.toscrape.com/catalogue/' + links[i].find('a').get('href'))
        #print(f'Done Scraping {categorys}')

scategory = set(category)
# Set up the connection
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='UseYourPass :D'
)

# Create a new cursor
cursor = conn.cursor()
# Drop the database if it already exists
cursor.execute("DROP DATABASE IF EXISTS BooksInfo")
# Create the database
cursor.execute('CREATE DATABASE BooksInfo;')
cursor.execute("USE BooksInfo;")
cursor.execute("""CREATE TABLE Categories(
                catID INT NOT NULL UNIQUE AUTO_INCREMENT,
                catName VARCHAR(255),

                PRIMARY KEY(catID));""")
cursor.execute("""CREATE TABLE Books (
                bookID INT NOT NULL AUTO_INCREMENT,
                bookName VARCHAR(255) NOT NULL,
                bookPrice DECIMAL(10, 2) NOT NULL,
                bookRating INT,
                catID INT,

                PRIMARY KEY (bookID),
                FOREIGN KEY (catID) REFERENCES categories(catID));""")

catDict = {}
id_counter = 1
for cat in scategory:
    catDict[cat] = id_counter
    id_counter += 1
for id_counter, cat in enumerate(scategory, start=1):
    cursor.execute('INSERT INTO Categories (catID, catName) VALUES (%s, %s)', (id_counter, cat))
    conn.commit()
rating_map = {'One': 1, 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5}
star = [rating_map[r] for r in star]
catID = [catDict[cat] for cat in category]
# Combine the lists into a list of tuples
books = list(zip(title, price, star, catID))
for value in books:
    #print(value)
    cursor.execute('INSERT INTO Books (bookName, bookPrice, bookRating, catID) VALUES (%s, %s, %s, %s)', value)
    conn.commit()
query = """

            SELECT *
            FROM Books as b INNER JOIN Categories as c 
            ON b.catID = c.catID;


"""
display(pd.read_sql(query, conn))
# conn.close()

/var/folders/y4/h840pn3s4knbxtcdr02w0kj40000gn/T/ipykernel_6367/2594684342.py:153: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(query, conn)


,bookID,bookName,bookPrice,bookRating,catID,catName
0,12,Sharp Objects,47.82,4,1,Mystery
1,13,"In a Dark, Dark Wood",19.63,1,1,Mystery
2,14,The Past Never Ends,56.50,4,1,Mystery
3,15,A Murder in Time,16.64,1,1,Mystery
4,16,The Murder of Roger Ackroyd (Hercule Poirot #4),44.10,4,1,Mystery
...,...,...,...,...,...,...
995,316,Counting Thyme,10.62,1,50,Childrens
996,317,Are We There Yet?,10.66,3,50,Childrens
997,318,Diary of a Minecraft Zombie Book 1: A Scare of...,52.88,4,50,Childrens
998,319,Matilda,28.34,1,50,Childrens


In [10]:
from IPython.display import display, Markdown

In [16]:
# create a function to display each question and its SQL query
def display_question(question_num, question_text, query):
    display(Markdown(f"**Q{question_num}: {question_text}**"))
    display(Markdown(f"```sql\n{query}\n```"))
    conn = mysql.connector.connect(
    host ='localhost',
    user ='root',
    password ='UseYourPass :D',
    database = 'BooksInfo'
)
    # execute the SQL query and display the results
    display(pd.read_sql(query, conn))

In [17]:
# call the display_question function for each question
display_question(1, "Write a sql query to get books that has rate >=3 and has 'Mr' in its name.",
                """
                SELECT *
                FROM Books
                WHERE bookRating >= 3 AND bookName LIKE '%Mr%';
""")

**Q1: Write a sql query to get books that has rate >=3 and has 'Mr' in its name.**

```sql

                SELECT *
                FROM Books
                WHERE bookRating >= 3 AND bookName LIKE '%Mr%';

```

/var/folders/y4/h840pn3s4knbxtcdr02w0kj40000gn/T/ipykernel_6367/2277814862.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql(query, conn))


,bookID,bookName,bookPrice,bookRating,catID
0,48,Mrs. Houdini,30.25,5,34
1,243,My Mrs. Brown,24.48,3,22
2,248,Dear Mr. Knightley,11.21,5,22


In [18]:
# call the display_question function for each question
display_question(2, "Write a sql query to get books that has rate >=3 or price > 20.",
                """
                SELECT *
                FROM Books
                WHERE bookRating >= 3 OR bookPrice > 20;
""")

**Q2: Write a sql query to get books that has rate >=3 or price > 20.**

```sql

                SELECT *
                FROM Books
                WHERE bookRating >= 3 OR bookPrice > 20;

```

/var/folders/y4/h840pn3s4knbxtcdr02w0kj40000gn/T/ipykernel_6367/2277814862.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql(query, conn))


,bookID,bookName,bookPrice,bookRating,catID
0,1,It's Only the Himalayas,45.17,2,26
1,2,Full Moon over Noah’s Ark: An Odyssey to Mount...,49.43,4,26
2,3,See America: A Celebration of Our National Par...,48.87,3,26
3,4,Vagabonding: An Uncommon Guide to the Art of L...,36.94,2,26
4,5,Under the Tuscan Sun,37.33,3,26
...,...,...,...,...,...
915,995,Libertarianism for Beginners,51.33,2,4
916,996,Why the Right Went Wrong: Conservatism--From G...,52.65,4,4
917,997,Equal Is Unfair: America's Misguided Fight Aga...,56.86,1,4
918,998,Amid the Chaos,36.58,1,11


In [19]:
# call the display_question function for each question
display_question(3, "Write a sql query to get books that has rate not 3.",
                """
                SELECT *
                FROM Books
                WHERE bookRating <> 3;
""")

**Q3: Write a sql query to get books that has rate not 3.**

```sql

                SELECT *
                FROM Books
                WHERE bookRating <> 3;

```

/var/folders/y4/h840pn3s4knbxtcdr02w0kj40000gn/T/ipykernel_6367/2277814862.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql(query, conn))


,bookID,bookName,bookPrice,bookRating,catID
0,1,It's Only the Himalayas,45.17,2,26
1,2,Full Moon over Noah’s Ark: An Odyssey to Mount...,49.43,4,26
2,4,Vagabonding: An Uncommon Guide to the Art of L...,36.94,2,26
3,6,A Summer In Europe,44.34,2,26
4,7,The Great Railway Bazaar,30.54,1,26
...,...,...,...,...,...
792,996,Why the Right Went Wrong: Conservatism--From G...,52.65,4,4
793,997,Equal Is Unfair: America's Misguided Fight Aga...,56.86,1,4
794,998,Amid the Chaos,36.58,1,11
795,999,Dark Notes,19.19,5,31


In [20]:
# call the display_question function for each question
display_question(4, "Write a sql query to get books that has price between 10 and 40 and has rate 3 or 4 or 1.",
                """
                SELECT *
                FROM Books
                WHERE bookPrice BETWEEN 10 AND 40
                    AND bookRating IN (1, 3, 4);
""")

**Q4: Write a sql query to get books that has price between 10 and 40 and has rate 3 or 4 or 1.**

```sql

                SELECT *
                FROM Books
                WHERE bookPrice BETWEEN 10 AND 40
                    AND bookRating IN (1, 3, 4);

```

/var/folders/y4/h840pn3s4knbxtcdr02w0kj40000gn/T/ipykernel_6367/2277814862.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql(query, conn))


,bookID,bookName,bookPrice,bookRating,catID
0,5,Under the Tuscan Sun,37.33,3,26
1,7,The Great Railway Bazaar,30.54,1,26
2,9,The Road to Little Dribbling: Adventures of an...,23.21,1,26
3,10,Neither Here nor There: Travels in Europe,38.95,3,26
4,13,"In a Dark, Dark Wood",19.63,1,1
...,...,...,...,...,...
350,984,The Girl You Left Behind (The Girl You Left Be...,15.79,1,12
351,987,Blue Like Jazz: Nonreligious Thoughts on Chris...,25.77,1,23
352,989,The Grownup,35.88,1,35
353,998,Amid the Chaos,36.58,1,11


In [21]:
# call the display_question function for each question
display_question(5, "Write a sql query to get the top 5 most expensive books (order books desc with price column and then limit the result to the first 5).",
                """
                SELECT *
                FROM Books
                ORDER BY bookPrice DESC
                LIMIT 5;
""")

**Q5: Write a sql query to get the top 5 most expensive books (order books desc with price column and then limit the result to the first 5).**

```sql

                SELECT *
                FROM Books
                ORDER BY bookPrice DESC
                LIMIT 5;

```

/var/folders/y4/h840pn3s4knbxtcdr02w0kj40000gn/T/ipykernel_6367/2277814862.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql(query, conn))


,bookID,bookName,bookPrice,bookRating,catID
0,192,The Perfect Play (Play by Play #1),59.99,3,6
1,272,Last One Home (New Beginnings #1),59.98,3,22
2,846,Civilization and Its Discontents,59.95,2,44
3,926,The Barefoot Contessa Cookbook,59.92,5,5
4,369,The Diary of a Young Girl,59.90,3,29


In [22]:
# call the display_question function for each question
display_question(6, "Write a sql query to get the 3rd 10 books in the books table order first by rate desc and then by price asc.",
                """
                SELECT *
                FROM Books
                ORDER BY bookRating DESC, bookPrice ASC
                LIMIT 20, 10;
""")

**Q6: Write a sql query to get the 3rd 10 books in the books table order first by rate desc and then by price asc.**

```sql

                SELECT *
                FROM Books
                ORDER BY bookRating DESC, bookPrice ASC
                LIMIT 20, 10;

```

/var/folders/y4/h840pn3s4knbxtcdr02w0kj40000gn/T/ipykernel_6367/2277814862.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql(query, conn))


,bookID,bookName,bookPrice,bookRating,catID
0,763,The Epidemic (The Program 0.6),14.44,5,41
1,189,A Gentleman's Position (Society of Gentlemen #3),14.75,5,6
2,862,"Hyperbole and a Half: Unfortunate Situations, ...",14.75,5,14
3,424,Disrupted: My Misadventure in the Start-Up Bubble,15.28,5,29
4,858,Fifty Shades Freed (Fifty Shades #3),15.36,5,15
5,94,Batman: The Dark Knight Returns (Batman),15.38,5,28
6,828,The Collected Poems of W.B. Yeats (The Collect...,15.42,5,17
7,780,Kill the Boy Band,15.52,5,41
8,164,Sophie's World,15.94,5,37
9,546,Shiver (The Wolves of Mercy Falls #1),16.23,5,39


    07) Write a sql query to add a new category in categories table and add 5 books to this category in books table with title, rate and price.

In [24]:
# Establish a connection to the MySQL database
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='UseYourPass :D',
    database='BooksInfo'
)

# Create a cursor object
cursor = conn.cursor()

# Add a new category to the Categories table
category_name = "New Category"
add_category_query = "INSERT INTO Categories (catName) VALUES (%s)"
cursor.execute(add_category_query, (category_name,))
conn.commit()

# Retrieve the last inserted category ID
last_category_id = cursor.lastrowid

In [27]:
# Add 5 books to the Books table with the corresponding category ID
books_data = [
    ("Book 1", 19.99, 4, last_category_id),
    ("Book 2", 15.99, 3, last_category_id),
    ("Book 3", 24.99, 5, last_category_id),
    ("Book 4", 29.99, 4, last_category_id),
    ("Book 5", 12.99, 2, last_category_id)
]
add_books_query = "INSERT INTO Books (bookName, bookPrice, bookRating, catID) VALUES (%s, %s, %s, %s)"
cursor.executemany(add_books_query, books_data)
conn.commit()

In [29]:
# call the display_question function for each question
display_question(7, "Write a sql query to add a new category in categories table and add 5 books to this category in books table with title, rate and price.",
                """
                SELECT Books.*, Categories.catName
                FROM Books
                INNER JOIN Categories ON Books.catID = Categories.catID
                WHERE Categories.catName = 'New Category';
""")

**Q7: Write a sql query to add a new category in categories table and add 5 books to this category in books table with title, rate and price.**

```sql

                SELECT Books.*, Categories.catName
                FROM Books
                INNER JOIN Categories ON Books.catID = Categories.catID
                WHERE Categories.catName = 'New Category';

```

/var/folders/y4/h840pn3s4knbxtcdr02w0kj40000gn/T/ipykernel_6367/2277814862.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql(query, conn))


,bookID,bookName,bookPrice,bookRating,catID,catName
0,1001,Book 1,19.99,4,51,New Category
1,1002,Book 2,15.99,3,51,New Category
2,1003,Book 3,24.99,5,51,New Category
3,1004,Book 4,29.99,4,51,New Category
4,1005,Book 5,12.99,2,51,New Category


	08) Write a sql query to update book's rate to 3 that thier price < 20£.

In [32]:
# call the display_question function for each question
display_question('8-1', "Before Update SQL query to get book's that their price < 20£.",
                """
                SELECT * FROM Books WHERE bookPrice < 20;
""")

**Q8-1: Before Update SQL query to get book's that their price < 20£.**

```sql

                SELECT * FROM Books WHERE bookPrice < 20;

```

/var/folders/y4/h840pn3s4knbxtcdr02w0kj40000gn/T/ipykernel_6367/2277814862.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql(query, conn))


,bookID,bookName,bookPrice,bookRating,catID
0,13,"In a Dark, Dark Wood",19.63,1,1
1,15,A Murder in Time,16.64,1,1
2,18,That Darkness (Gardiner and Renner #1),13.92,1,1
3,19,Tastes Like Fear (DI Marnie Rome #3),10.69,1,1
4,21,A Study in Scarlet (Sherlock Holmes #1),16.73,2,1
...,...,...,...,...,...
194,999,Dark Notes,19.19,5,31
195,1000,The Long Shadow of Small Ghosts: Murder and Me...,10.97,1,18
196,1001,Book 1,19.99,4,51
197,1002,Book 2,15.99,3,51


In [33]:
# Establish a connection to the MySQL database
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='UseYourPass :D',
    database='BooksInfo'
)

# Create a cursor object
cursor = conn.cursor()

# Update the book ratings
cursor.execute("""UPDATE Books
                  SET bookRating = 3
                  WHERE bookPrice < 20;""")

conn.commit()

In [34]:
# call the display_question function for each question
display_question('8-2', "After Update SQL query to get book's that their price < 20£ with Updated rate 3.",
                """
                SELECT * FROM Books WHERE bookPrice < 20;
""")

**Q8-2: After Update SQL query to get book's that their price < 20£ with Updated rate 3.**

```sql

                SELECT * FROM Books WHERE bookPrice < 20;

```

/var/folders/y4/h840pn3s4knbxtcdr02w0kj40000gn/T/ipykernel_6367/2277814862.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql(query, conn))


,bookID,bookName,bookPrice,bookRating,catID
0,13,"In a Dark, Dark Wood",19.63,3,1
1,15,A Murder in Time,16.64,3,1
2,18,That Darkness (Gardiner and Renner #1),13.92,3,1
3,19,Tastes Like Fear (DI Marnie Rome #3),10.69,3,1
4,21,A Study in Scarlet (Sherlock Holmes #1),16.73,3,1
...,...,...,...,...,...
194,999,Dark Notes,19.19,3,31
195,1000,The Long Shadow of Small Ghosts: Murder and Me...,10.97,3,18
196,1001,Book 1,19.99,3,51
197,1002,Book 2,15.99,3,51


	09) Write a sql query to delete all books that have price > 50£ and has rate <= 2.

In [35]:
# call the display_question function for each question
display_question('9-1', "Before Write a sql query to delete all books that have price > 50£ and has rate <= 2.",
                """
                SELECT *
                FROM Books
                WHERE bookPrice > 50 AND bookRating <= 2;
""")

**Q9-1: Before Write a sql query to delete all books that have price > 50£ and has rate <= 2.**

```sql

                SELECT *
                FROM Books
                WHERE bookPrice > 50 AND bookRating <= 2;

```

/var/folders/y4/h840pn3s4knbxtcdr02w0kj40000gn/T/ipykernel_6367/2277814862.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql(query, conn))


,bookID,bookName,bookPrice,bookRating,catID
0,17,The Last Mile (Amos Decker #2),54.21,2,1
1,43,1st to Die (Women's Murder Club #1),53.98,1,1
2,44,Tipping the Velvet,53.74,1,34
3,61,The Last Painting of Sara de Vos,55.55,2,34
4,74,"Saga, Volume 5 (Saga (Collected Editions) #5)",51.04,2,28
...,...,...,...,...,...
69,922,32 Yolks,53.63,2,5
70,949,"Rich Dad, Poor Dad",51.74,1,3
71,992,"Eat Fat, Get Thin",54.07,2,8
72,995,Libertarianism for Beginners,51.33,2,4


In [36]:
# Establish a connection to the MySQL database
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='UseYourPass :D',
    database='BooksInfo'
)

# Create a cursor object
cursor = conn.cursor()

# Execute the delete query
cursor.execute("""DELETE FROM Books
                    WHERE bookPrice > 50
                    AND
                    bookRating <= 2""")
conn.commit()

In [37]:
# call the display_question function for each question
display_question('9-1', "After Write a sql query to delete all books that have price > 50£ and has rate <= 2.",
                """
                SELECT *
                FROM Books
                WHERE bookPrice > 50 AND bookRating <= 2;
""")

**Q9-1: After Write a sql query to delete all books that have price > 50£ and has rate <= 2.**

```sql

                SELECT *
                FROM Books
                WHERE bookPrice > 50 AND bookRating <= 2;

```

/var/folders/y4/h840pn3s4knbxtcdr02w0kj40000gn/T/ipykernel_6367/2277814862.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql(query, conn))


,bookID,bookName,bookPrice,bookRating,catID


In [38]:
# call the display_question function for each question
display_question(10, "Write a sql query to count the number of books that have 'Secret' in thier names and price between 10£ and 25£.",
                """
                SELECT COUNT(*) AS bookCount
                FROM Books
                WHERE bookName LIKE '%Secret%' AND bookPrice BETWEEN 10 AND 25;
""")

**Q10: Write a sql query to count the number of books that have 'Secret' in thier names and price between 10£ and 25£.**

```sql

                SELECT COUNT(*) AS bookCount
                FROM Books
                WHERE bookName LIKE '%Secret%' AND bookPrice BETWEEN 10 AND 25;

```

/var/folders/y4/h840pn3s4knbxtcdr02w0kj40000gn/T/ipykernel_6367/2277814862.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql(query, conn))


,bookCount
0,5


In [39]:
# call the display_question function for each question
display_question(11, "Write a sql query to get the minimum & maximum price for all the books that have rate 5.",
                """
                SELECT MIN(bookPrice) AS minPrice, MAX(bookPrice) AS maxPrice
                FROM Books
                WHERE bookRating = 5;
""")

**Q11: Write a sql query to get the minimum & maximum price for all the books that have rate 5.**

```sql

                SELECT MIN(bookPrice) AS minPrice, MAX(bookPrice) AS maxPrice
                FROM Books
                WHERE bookRating = 5;

```

/var/folders/y4/h840pn3s4knbxtcdr02w0kj40000gn/T/ipykernel_6367/2277814862.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql(query, conn))


,minPrice,maxPrice
0,20.02,59.92


In [40]:
# call the display_question function for each question
display_question(12, "Write a sql query to calculate the avg price for all the books that have rate 5.",
                """
                SELECT ROUND(AVG(bookPrice), 2) AS avgPrice
                FROM Books
                WHERE bookRating = 5;
""")

**Q12: Write a sql query to calculate the avg price for all the books that have rate 5.**

```sql

                SELECT ROUND(AVG(bookPrice), 2) AS avgPrice
                FROM Books
                WHERE bookRating = 5;

```

/var/folders/y4/h840pn3s4knbxtcdr02w0kj40000gn/T/ipykernel_6367/2277814862.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql(query, conn))


,avgPrice
0,40.95


In [41]:
# call the display_question function for each question
display_question(13, "Write a sql query to sum all book's price that have rate 2 and price between 10 and 40.",
                """
                SELECT SUM(bookPrice) AS totalSum
                FROM Books
                WHERE bookRating = 2
                AND bookPrice BETWEEN 10 AND 40;
""")

**Q13: Write a sql query to sum all book's price that have rate 2 and price between 10 and 40.**

```sql

                SELECT SUM(bookPrice) AS totalSum
                FROM Books
                WHERE bookRating = 2
                AND bookPrice BETWEEN 10 AND 40;

```

/var/folders/y4/h840pn3s4knbxtcdr02w0kj40000gn/T/ipykernel_6367/2277814862.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql(query, conn))


,totalSum
0,2772.75


In [42]:
# call the display_question function for each question
display_question(14, "Write a sql query to join both books & categories table into one new table containing book_name & category_name & book_rate and book_price.",
                """
                SELECT b.bookName, c.catName, b.bookRating, b.bookPrice
                FROM Books b
                JOIN Categories c ON b.catID = c.catID;
""")

**Q14: Write a sql query to join both books & categories table into one new table containing book_name & category_name & book_rate and book_price.**

```sql

                SELECT b.bookName, c.catName, b.bookRating, b.bookPrice
                FROM Books b
                JOIN Categories c ON b.catID = c.catID;

```

/var/folders/y4/h840pn3s4knbxtcdr02w0kj40000gn/T/ipykernel_6367/2277814862.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql(query, conn))


,bookName,catName,bookRating,bookPrice
0,Sharp Objects,Mystery,4,47.82
1,"In a Dark, Dark Wood",Mystery,3,19.63
2,The Past Never Ends,Mystery,4,56.50
3,A Murder in Time,Mystery,3,16.64
4,The Murder of Roger Ackroyd (Hercule Poirot #4),Mystery,4,44.10
...,...,...,...,...
926,Book 1,New Category,3,19.99
927,Book 2,New Category,3,15.99
928,Book 3,New Category,5,24.99
929,Book 4,New Category,4,29.99


In [43]:
# call the display_question function for each question
display_question(15, "Write a sql query to calculate how many books each rate has and have price between 20£ and 30£.",
                """
                SELECT bookRating, COUNT(*) AS bookCount
                FROM Books
                WHERE bookPrice BETWEEN 20 AND 30
                GROUP BY bookRating
                ORDER BY bookRating;
""")

**Q15: Write a sql query to calculate how many books each rate has and have price between 20£ and 30£.**

```sql

                SELECT bookRating, COUNT(*) AS bookCount
                FROM Books
                WHERE bookPrice BETWEEN 20 AND 30
                GROUP BY bookRating
                ORDER BY bookRating;

```

/var/folders/y4/h840pn3s4knbxtcdr02w0kj40000gn/T/ipykernel_6367/2277814862.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql(query, conn))


,bookRating,bookCount
0,1,56
1,2,40
2,3,41
3,4,35
4,5,37


In [44]:
# call the display_question function for each question
display_question(16, "Write a sql query to calculate how many books each category has having count > 10.",
                """
                SELECT c.catName, COUNT(*) AS bookCount
                FROM Books b
                JOIN Categories c ON b.catID = c.catID
                GROUP BY c.catName
                HAVING COUNT(*) > 10;
""")

**Q16: Write a sql query to calculate how many books each category has having count > 10.**

```sql

                SELECT c.catName, COUNT(*) AS bookCount
                FROM Books b
                JOIN Categories c ON b.catID = c.catID
                GROUP BY c.catName
                HAVING COUNT(*) > 10;

```

/var/folders/y4/h840pn3s4knbxtcdr02w0kj40000gn/T/ipykernel_6367/2277814862.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql(query, conn))


,catName,bookCount
0,Mystery,30
1,Add a comment,60
2,Business,11
3,Food and Drink,28
4,Romance,33
5,Womens Fiction,16
6,History,16
7,Poetry,18
8,Fantasy,40
9,Fiction,60


In [45]:
# call the display_question function for each question
display_question(17, "Write a sql query to get all books with category_name='Music' using subquery.",
                """
                SELECT bookName
                FROM Books
                WHERE catID = (
                    SELECT catID
                    FROM Categories
                    WHERE catName = 'Music'
                );
""")

**Q17: Write a sql query to get all books with category_name='Music' using subquery.**

```sql

                SELECT bookName
                FROM Books
                WHERE catID = (
                    SELECT catID
                    FROM Categories
                    WHERE catName = 'Music'
                );

```

/var/folders/y4/h840pn3s4knbxtcdr02w0kj40000gn/T/ipykernel_6367/2277814862.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql(query, conn))


,bookName
0,Rip it Up and Start Again
1,Our Band Could Be Your Life: Scenes from the A...
2,How Music Works
3,Love Is a Mix Tape (Music #1)
4,Please Kill Me: The Uncensored Oral History of...
5,Kill 'Em and Leave: Searching for James Brown ...
6,This Is Your Brain on Music: The Science of a ...
7,Orchestra of Exiles: The Story of Bronislaw Hu...
8,No One Here Gets Out Alive
9,Life


In [46]:
# call the display_question function for each question
display_question(17, "Write a sql query to get all books with category_name='Music' using subquery.",
                """
                SELECT b.bookName, (
                    SELECT catName
                    FROM Categories
                    WHERE catID = b.catID
                ) AS catName
                FROM Books b
                WHERE b.catID = (
                    SELECT catID
                    FROM Categories
                    WHERE catName = 'Music'
                );
""")

**Q17: Write a sql query to get all books with category_name='Music' using subquery.**

```sql

                SELECT b.bookName, (
                    SELECT catName
                    FROM Categories
                    WHERE catID = b.catID
                ) AS catName
                FROM Books b
                WHERE b.catID = (
                    SELECT catID
                    FROM Categories
                    WHERE catName = 'Music'
                );

```

/var/folders/y4/h840pn3s4knbxtcdr02w0kj40000gn/T/ipykernel_6367/2277814862.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql(query, conn))


,bookName,catName
0,Rip it Up and Start Again,Music
1,Our Band Could Be Your Life: Scenes from the A...,Music
2,How Music Works,Music
3,Love Is a Mix Tape (Music #1),Music
4,Please Kill Me: The Uncensored Oral History of...,Music
5,Kill 'Em and Leave: Searching for James Brown ...,Music
6,This Is Your Brain on Music: The Science of a ...,Music
7,Orchestra of Exiles: The Story of Bronislaw Hu...,Music
8,No One Here Gets Out Alive,Music
9,Life,Music


In [47]:
conn.close()

<details><summary>Done By</summary>
<center> Ahmed NasrElDin </center> 
</details>
